In [18]:
from datasets import load_dataset,load_from_disk


In [19]:
ds = load_from_disk("/Users/zzzsy/project/RLHF/code/llama3/precomputed_traindataset")
ds_score = load_from_disk("/Users/zzzsy/project/RLHF/code/llama3/test_data/traindataset")

In [20]:
print(ds)
print(ds_score)

Dataset({
    features: ['prompt', 'chosen', 'rejected', 'ref_logp_chosen', 'ref_logp_rejected', 'reward_chosen', 'reward_rejected'],
    num_rows: 60038
})
Dataset({
    features: ['prompt', 'all_generated_responses', 'all_rm_scores', 'chosen', 'rejected'],
    num_rows: 60038
})


In [21]:
import numpy as np

def compute_f_score_loo(example):
    # 获取原始分数列表 (通常长度为 5)
    scores = example['all_rm_scores']
    
    # 1. 确定 Target (Y2) 的分数
    r_chosen = np.max(scores)
    r_rejected = np.min(scores)
    
    # 2. 构建 Reference (Y1) 列表 (Leave-One-Out)
    
    # --- 计算 f_chosen ---
    # 复制列表以防修改原数据
    scores_for_chosen = list(scores)
    # 移除 chosen 的分数，剩下的作为 Y1
    scores_for_chosen.remove(r_chosen)
    # 转换为 numpy 数组以便计算
    y1_chosen_array = np.array(scores_for_chosen)
    
    # 公式: E [ exp(r1) / (exp(r1) + exp(r_chosen)) ]
    # 这里 r1 来自 y1_chosen_array (4个元素)
    num_c = np.exp(y1_chosen_array)
    den_c = num_c + np.exp(r_chosen)
    f_chosen = np.mean(num_c / den_c)
    
    # --- 计算 f_rejected ---
    scores_for_rejected = list(scores)
    # 移除 rejected 的分数，剩下的作为 Y1
    scores_for_rejected.remove(r_rejected)
    y1_rejected_array = np.array(scores_for_rejected)
    
    # 公式: E [ exp(r1) / (exp(r1) + exp(r_rejected)) ]
    num_r = np.exp(y1_rejected_array)
    den_r = num_r + np.exp(r_rejected)
    f_rejected = np.mean(num_r / den_r)
    
    return {
        "f_chosen": f_chosen,
        "f_rejected": f_rejected
    }

# 应用到数据集
ds_with_f = ds_score.map(compute_f_score_loo, num_proc=4)

# 验证结果
print("Prompt:", ds_with_f[0]['prompt'][:50] + "...")
print("Scores:", ds_with_f[0]['all_rm_scores'])
print("f_chosen (Expected Low):", ds_with_f[0]['f_chosen'])
print("f_rejected (Expected High):", ds_with_f[0]['f_rejected'])

Prompt: 

A federal judge on Thursday rejected efforts by ...
Scores: [0.11962890625, 0.11962890625, 0.1201171875, 0.12158203125, 0.12109375]
f_chosen (Expected Low): 0.4996337891570874
f_rejected (Expected High): 0.5002441405692177


In [22]:
print(ds[0])

{'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nA federal judge on Thursday rejected efforts by Texas to stop the resettlement of Syrian refugees within its borders. The ruling comes days after presumptive Republican presidential nominee Donald Trump called to block non-citizen Muslims from entering the U.S. following the Orlando nightclub massacre. Even before the attack, Americans were divided, mostly along party lines, on the issue of refugees from war-torn countries in the Middle East. A survey conducted in late May by the Brookings Institution found of those polled, just 38 percent of Republicans supported taking in refugees from Syria and the Middle East, compared with 77 percent of Democrats. But among Trump supporters, an overwhelming 77 percent said they oppose taking in refugees. So who are these people who cause such a divide among Americans, triggering feelings of dread and suspicion in some and feelings of empathy and hospitality in others? Million

In [23]:
# 1. 从计算好的数据集中提取列表
f_chosen_values = ds_with_f['f_chosen']
f_rejected_values = ds_with_f['f_rejected']

# 2. 将列表作为新列添加到 ds_new 中
# 注意：HuggingFace 的 Dataset 是不可变的，add_column 会返回一个新的对象
ds_final = ds.add_column("f_chosen", f_chosen_values)
ds_final = ds_final.add_column("f_rejected", f_rejected_values)

# 3. 验证结果
print(ds_final)
# 预期输出应该包含新列：
# features: ['prompt', 'chosen', 'rejected', 'ref_logp_chosen', ..., 'f_chosen', 'f_rejected']

Dataset({
    features: ['prompt', 'chosen', 'rejected', 'ref_logp_chosen', 'ref_logp_rejected', 'reward_chosen', 'reward_rejected', 'f_chosen', 'f_rejected'],
    num_rows: 60038
})


In [24]:
print(ds_final[0])

{'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nA federal judge on Thursday rejected efforts by Texas to stop the resettlement of Syrian refugees within its borders. The ruling comes days after presumptive Republican presidential nominee Donald Trump called to block non-citizen Muslims from entering the U.S. following the Orlando nightclub massacre. Even before the attack, Americans were divided, mostly along party lines, on the issue of refugees from war-torn countries in the Middle East. A survey conducted in late May by the Brookings Institution found of those polled, just 38 percent of Republicans supported taking in refugees from Syria and the Middle East, compared with 77 percent of Democrats. But among Trump supporters, an overwhelming 77 percent said they oppose taking in refugees. So who are these people who cause such a divide among Americans, triggering feelings of dread and suspicion in some and feelings of empathy and hospitality in others? Million

In [25]:
import logging
from datasets import disable_progress_bar

In [26]:
print("开始保存数据...")  # <--- 这行能让你确信代码开始运行了

# 定义保存路径
save_path = "/Users/zzzsy/project/RLHF/code/llama3/taindataset_with_f"

# 保存数据
# 这一步可能需要几分钟，取决于硬盘速度和数据大小
ds_final.save_to_disk(save_path)

print(f"✅ 数据已成功保存至: {save_path}")

开始保存数据...


Saving the dataset (1/1 shards): 100%|██████████| 60038/60038 [00:00<00:00, 69381.22 examples/s]

✅ 数据已成功保存至: /Users/zzzsy/project/RLHF/code/llama3/taindataset_with_f


In [28]:
ds_f = load_from_disk("/Users/zzzsy/project/RLHF/code/llama3/taindataset_with_f")
print(ds_f[0])
print(ds_f)

{'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nA federal judge on Thursday rejected efforts by Texas to stop the resettlement of Syrian refugees within its borders. The ruling comes days after presumptive Republican presidential nominee Donald Trump called to block non-citizen Muslims from entering the U.S. following the Orlando nightclub massacre. Even before the attack, Americans were divided, mostly along party lines, on the issue of refugees from war-torn countries in the Middle East. A survey conducted in late May by the Brookings Institution found of those polled, just 38 percent of Republicans supported taking in refugees from Syria and the Middle East, compared with 77 percent of Democrats. But among Trump supporters, an overwhelming 77 percent said they oppose taking in refugees. So who are these people who cause such a divide among Americans, triggering feelings of dread and suspicion in some and feelings of empathy and hospitality in others? Million

In [1]:
from datasets import load_dataset,load_from_disk
ds_simpo = load_dataset("princeton-nlp/llama3-ultrafeedback-armorm")


/opt/anaconda3/envs/rlhf/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Generating test split: 100%|██████████| 1961/

In [2]:
print(ds_simpo)

DatasetDict({
    train: Dataset({
        features: ['prompt_id', 'prompt', 'all_generated_responses', 'all_rm_scores', 'chosen', 'rejected'],
        num_rows: 59876
    })
    test: Dataset({
        features: ['prompt_id', 'prompt', 'all_generated_responses', 'all_rm_scores', 'chosen', 'rejected'],
        num_rows: 1961
    })
})


In [3]:
import numpy as np

def compute_f_score_loo(example):
    # 获取原始分数列表 (通常长度为 5)
    scores = example['all_rm_scores']
    
    # 1. 确定 Target (Y2) 的分数
    r_chosen = np.max(scores)
    r_rejected = np.min(scores)
    
    # 2. 构建 Reference (Y1) 列表 (Leave-One-Out)
    
    # --- 计算 f_chosen ---
    # 复制列表以防修改原数据
    scores_for_chosen = list(scores)
    # 移除 chosen 的分数，剩下的作为 Y1
    scores_for_chosen.remove(r_chosen)
    # 转换为 numpy 数组以便计算
    y1_chosen_array = np.array(scores_for_chosen)
    
    # 公式: E [ exp(r1) / (exp(r1) + exp(r_chosen)) ]
    # 这里 r1 来自 y1_chosen_array (4个元素)
    num_c = np.exp(y1_chosen_array)
    den_c = num_c + np.exp(r_chosen)
    f_chosen = np.mean(num_c / den_c)
    
    # --- 计算 f_rejected ---
    scores_for_rejected = list(scores)
    # 移除 rejected 的分数，剩下的作为 Y1
    scores_for_rejected.remove(r_rejected)
    y1_rejected_array = np.array(scores_for_rejected)
    
    # 公式: E [ exp(r1) / (exp(r1) + exp(r_rejected)) ]
    num_r = np.exp(y1_rejected_array)
    den_r = num_r + np.exp(r_rejected)
    f_rejected = np.mean(num_r / den_r)
    
    return {
        "f_chosen": f_chosen,
        "f_rejected": f_rejected
    }

# 应用到数据集
ds_with_f = ds_simpo.map(compute_f_score_loo, num_proc=4)

# 验证结果
print("Prompt:", ds_with_f['train'][0]['prompt'][:50] + "...")
print("Scores:", ds_with_f['train'][0]['all_rm_scores'])
print("f_chosen (Expected Low):", ds_with_f['train'][0]['f_chosen'])
print("f_rejected (Expected High):", ds_with_f['train'][0]['f_rejected'])

Map (num_proc=4): 100%|██████████| 1961/1961 [00:00<00:00, 13996.43 examples/s]

Prompt: how can i develop a habit of drawing daily...
Scores: [0.205078125, 0.2021484375, 0.201171875, 0.2099609375, 0.208984375]
f_chosen (Expected Low): 0.49859619803702876
f_rejected (Expected High): 0.5013427671026092


In [4]:
f_chosen_values = ds_with_f['train']['f_chosen']
f_rejected_values = ds_with_f['train']['f_rejected']

# 2. 将列表作为新列添加到 ds_new 中
# 注意：HuggingFace 的 Dataset 是不可变的，add_column 会返回一个新的对象
ds_final = ds_simpo['train'].add_column("f_chosen", f_chosen_values)
ds_final = ds_final.add_column("f_rejected", f_rejected_values)

# 3. 验证结果
print(ds_final)

Dataset({
    features: ['prompt_id', 'prompt', 'all_generated_responses', 'all_rm_scores', 'chosen', 'rejected', 'f_chosen', 'f_rejected'],
    num_rows: 59876
})


In [5]:
print("开始保存数据...")  # <--- 这行能让你确信代码开始运行了

# 定义保存路径
save_path = "/Users/zzzsy/project/RLHF/code/llama3/sim_taindataset_with_f"

# 保存数据
# 这一步可能需要几分钟，取决于硬盘速度和数据大小
ds_final.save_to_disk(save_path)

print(f"✅ 数据已成功保存至: {save_path}")

开始保存数据...


Saving the dataset (2/2 shards): 100%|██████████| 59876/59876 [00:02<00:00, 27584.53 examples/s]

✅ 数据已成功保存至: /Users/zzzsy/project/RLHF/code/llama3/sim_taindataset_with_f
